In [1]:
import math
import numpy as np
import pandas as pd
import random
import torch
from attack import (
    reconstruct_interactions,
)
from dataset import (
    LearningToRankDataset,
)
from more_itertools import grouper
from ranker import (
    LinearPDGDRanker,
    Neural1LayerPDGDRanker,
)
from tqdm.notebook import tqdm
from utils import (
    CascadeClickModel,
    Metrics,
    apply_gaussian_mechanism,
    LtrEvaluator,
)

In [ ]:
def set_seed(seed=2023):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

# Change the dataset here
# Possible values: MQ2007, MSLR-WEB10K.
# (MQ2008 is also possible but it's a smaller dataset than MQ2007 and the results are very similar.)
dataset = "MQ2007"

if dataset == "MSLR-WEB10K":
    data = LearningToRankDataset("../dataset/MSLR-WEB10K/Fold1/train.txt", normalize=True)
    click_models = {
        "navigational": CascadeClickModel(prob_click=[0.05, 0.3, 0.5, 0.7, 0.95], prob_stop=[0.2, 0.3, 0.5, 0.7, 0.9]),
        "informational": CascadeClickModel(prob_click=[0.4, 0.6, 0.7, 0.8, 0.9], prob_stop=[0.1, 0.2, 0.3, 0.4, 0.5]),
    }
    # num_query_per_user = [12, 24, 48]
    num_query_per_user = [48] # Scaled down for artifact eval
else:
    data = LearningToRankDataset(f"../dataset/{dataset}/Fold1/train.txt", normalize=False)
    click_models = {
        "navigational": CascadeClickModel(prob_click=[0.05, 0.5, 0.95], prob_stop=[0.2, 0.5, 0.9]),
        "informational": CascadeClickModel(prob_click=[0.4, 0.7, 0.9], prob_stop=[0.1, 0.3, 0.5]),
    }
    # num_query_per_user = [4, 8, 16]
    num_query_per_user = [16] # Scaled down for artifact eval

num_features = data.get_num_features()
models = {
    "linear_pdgd": LinearPDGDRanker(num_features),
    # "neural_4_pdgd": Neural1LayerPDGDRanker(num_features, hidden_size=4),
    # "neural_8_pdgd": Neural1LayerPDGDRanker(num_features, hidden_size=8),
    "neural_16_pdgd": Neural1LayerPDGDRanker(num_features, hidden_size=16),
}

In [ ]:
# Main results in Table VII of Section VI.B as well as DP results in Table IX of Section VII.A

set_seed()

# Local learning parameters
num_item_per_ranking = 10
local_lr = 1e-01
num_sim_round = 1

# Stealthiness parameter: Control % of features to manipulate. Selection is random.
# alpha=1.0 means all features will be manipulated, 0.0 means no manipulation
# alphas = [0.0, 0.5, 0.75, 1.0]
alphas = [0.0, 1.0]

# Reconstruction attack parameters
num_atk = 1
max_iter = 1000
atk_lr = 0.1

# Differential privacy parameters
epsilons = [1.0, 20.0, 100.0, 500.0, math.inf]
delta = 1e-08
sensitivity = 0.5

metrics = Metrics()

# Local training algorithm
def train(model, params, grouped_train_data, local_lr=local_lr):
    cur_params = params.clone()

    for features, ranking, interactions in grouped_train_data:
        cur_grad = model.grad(
            cur_params,
            features,
            ranking,
            interactions,
        )
        cur_params = cur_params + local_lr * cur_grad

    return cur_params - params

def simulate_attack(model, model_name, grouped_data, click_model, epsilons, click_model_name, num_query):
    params = model.gen_params()
    indices = []
    start_ind = 0
    grouped_train_data_dict = {
        alpha: [] for alpha in alphas
    }

    # Sample clicks and prepare manipulated data
    for relevances, features in grouped_data:
        if len(relevances) == 1:
            continue
        features = torch.Tensor(features)
        ranking = model.rank(params, features, sample=True)[:num_item_per_ranking]
        interactions = torch.Tensor(click_model.click(ranking, relevances)) # Ground truth
        
        features = features[ranking]
        # Remap the original ranking into the correct range
        _, ranking = torch.where(
            torch.sort(ranking)[0].unsqueeze(1) == ranking.unsqueeze(0)
        )
        num_items = len(ranking)        
        noise = torch.normal(0.0, 0.1, features.shape)
        for alpha in alphas:
            mask = torch.ones_like(features)
            selected_features = random.sample(list(range(num_features)), int(num_features * alpha))
            mask[:, selected_features] = 0.0
            features_adm = mask * features + (1.0 - mask) * noise
            grouped_train_data_dict[alpha].append((features_adm, ranking, interactions))
        
        indices.append((start_ind, start_ind + num_items))
        start_ind += num_items

    if len(grouped_train_data_dict[alphas[0]]) < 1:
        return
    
    # Local training
    raw_target_dict = {
        key: train(
            model,
            params,
            random.sample(train_data, len(train_data)),
            local_lr,
        ) for key, train_data in grouped_train_data_dict.items()
    }

    for epsilon in epsilons:
        for key, raw_target in raw_target_dict.items():
            target = apply_gaussian_mechanism(raw_target, epsilon, delta, sensitivity)
            # Reconstruct
            train_data = grouped_train_data_dict[key]
            preds_raw, _ = reconstruct_interactions(
                lambda I: (train(
                    model,
                    params,
                    [
                        (features, ranking, I[indices[idx][0] : indices[idx][1]])
                        for idx, (features, ranking, _) in enumerate(train_data)
                    ],
                    local_lr,
                )) / local_lr,
                target / local_lr,
                indices[-1][1],
                lr=atk_lr,
                max_iter=max_iter,
                num_rounds=num_atk,
                return_raw=True,
            )
            preds = preds_raw.sigmoid().round().long()
            interactions = torch.cat([I for (_, _, I) in train_data]) # Ground truth

            metrics.update(
                f"{model_name}_{click_model_name}_{num_query}_query_eps_{epsilon}_{key}",
                interactions,
                preds,
                preds_raw=preds_raw,
            )

for _ in tqdm(range(num_sim_round)):
    query_ids = data.get_all_query_ids()
    query_ids = random.sample(query_ids, len(query_ids))

    for num_query in num_query_per_user:
        print("Num query", num_query)
        for qids in tqdm(grouper(query_ids, num_query, incomplete="ignore"), total=len(query_ids)//num_query):
            grouped_data = data.get_data_for_queries(list(qids))
            for model_name, model in models.items():
                for click_model_name, click_model in click_models.items():
                    simulate_attack(model, model_name, grouped_data, click_model, epsilons, click_model_name, num_query)

metrics.print_summary()
metrics.save("../output/ltr_metrics.csv")

In [ ]:
# Utility (NDCG) vs DP and number of users attacked
set_seed()

test_data = LearningToRankDataset(f"../dataset/{dataset}/Fold1/test.txt", normalize=True)

num_rounds = 10
num_query_per_user = [1]
num_item_per_ranking = 10
local_lr = 1e-01
epsilons = [1.0, 20.0, 100.0, 500.0, math.inf]
delta = 1e-08
sensitivity = 0.5
num_users_per_agg = 100
num_attack_users = [0, 5, 10, 20, 40, 60, 80, 90, 95, 100]

results = pd.DataFrame({
    "model_name": [],
    "click_model": [],
    "epsilon": [],
    "n_attacked_users": [],
    "ndcg": [],    
})
evaluator = LtrEvaluator(test_data, num_item_per_ranking)

query_ids = data.get_all_query_ids()

for model_name, model in models.items():
    for click_model_name, click_model in click_models.items():
        orig_model_params = model.gen_params()
        for n_users in num_attack_users:
            for epsilon in epsilons:
                print(f"Model: {model_name} | Click model: {click_model_name} | Epsilon: {epsilon} | Num attacked users: {n_users}")
                model_params = torch.clone(orig_model_params)
                ndcgs = []
                ndcgs.append(evaluator.calculate_average_offline_ndcg(model, model_params))

                for _ in tqdm(range(num_rounds)):
                    query_ids = random.sample(query_ids, len(query_ids))
                    grad_arr = []
                    n_users_to_attack = n_users

                    for qid in query_ids:
                        relevances, features = data.get_data_for_queries([qid])[0]

                        features = torch.Tensor(features)
                        ranking = model.rank(model_params, features, sample=True)[:num_item_per_ranking]
                        clicks = click_model.click(ranking, relevances, filter_all_or_zero=False)
                        if not np.any(clicks) or np.all(clicks):
                            continue
                        interactions = torch.Tensor(clicks)
                        features = features[ranking]

                        if n_users_to_attack > 0:
                            features = torch.normal(0, 0.1, features.shape)
                            n_users_to_attack -= 1

                        # Remap the original ranking into the correct range
                        _, ranking = torch.where(
                            torch.sort(ranking)[0].unsqueeze(1) == ranking.unsqueeze(0)
                        )
                        
                        raw_grad = local_lr * model.grad(
                            model_params,
                            features,
                            ranking,
                            interactions,
                        )

                        grad_arr.append(apply_gaussian_mechanism(raw_grad, epsilon, delta, sensitivity))

                        if (len(grad_arr) == num_users_per_agg):
                            model_params = model_params + torch.stack(grad_arr).mean(dim=0)
                            grad_arr = []
                            n_users_to_attack = n_users

                    if (len(grad_arr) > 0):
                        model_params = model_params + torch.stack(grad_arr).mean(dim=0)
                    
                    ndcgs.append(evaluator.calculate_average_offline_ndcg(model, model_params))
                    
                results.loc[len(results.index), :] = {
                    "model_name": model_name,
                    "click_model": click_model_name,
                    "epsilon": epsilon,
                    "n_attacked_users": n_users,
                    "ndcg": ndcgs[-1]
                }

                print(f"NDCG: {ndcgs[-1]}")

print(results.groupby(["model_name", "click_model", "epsilon", "n_attacked_users"]).describe().to_string())
results.to_csv("../output/ltr_utility.csv", index=False)

In [ ]:
# Secure Aggregration + LDP (Table XI of Section VII.B)

set_seed()

# Local training parameters
num_query_per_user = [4]
num_item_per_ranking = 10
local_lr = 1e-01
num_sim_round = 1

# Reconstruction attack parameters
num_atk = 1
max_iter = 1000
atk_lr = 0.1

num_users = [10, 100, 500, 1000]
epsilons = [700.0, 500.0, 300.0, 100.0]
delta = 1e-08
sensitivity = 0.5

metrics = Metrics()

def train(model, params, grouped_train_data, local_lr=local_lr):
    cur_params = params.clone()

    for features, ranking, interactions in grouped_train_data:
        cur_grad = model.grad(
            cur_params,
            features,
            ranking,
            interactions,
        )

        cur_params = cur_params + local_lr * cur_grad

    return cur_params - params

def simulate_attack(model, model_name, grouped_data, click_model, epsilons, click_model_name, num_query):
    params = model.gen_params()
    indices = []
    start_ind = 0
    grouped_train_data_dict = {
        "sa": []
    }

    for relevances, features in grouped_data:
        if len(relevances) == 1:
            continue
        features = torch.Tensor(features)
        ranking = model.rank(params, features, sample=True)[:num_item_per_ranking]
        interactions = torch.Tensor(click_model.click(ranking, relevances))
        
        features = features[ranking]
        # Remap the original ranking into the correct range
        _, ranking = torch.where(
            torch.sort(ranking)[0].unsqueeze(1) == ranking.unsqueeze(0)
        )
        num_items = len(ranking)
        features_adm = torch.normal(0.0, 0.1, features.shape)
        grouped_train_data_dict["sa"].append((features_adm, ranking, interactions))
        
        indices.append((start_ind, start_ind + num_items))
        start_ind += num_items

    raw_target_dict = {
        key: train(
            model,
            params,
            random.sample(train_data, len(train_data)),
            local_lr,
        ) for key, train_data in grouped_train_data_dict.items()
    }

    for epsilon in epsilons:
        for num_user in num_users:
            for key, raw_target in raw_target_dict.items():
                target = apply_gaussian_mechanism(raw_target, epsilon, delta, sensitivity)
                zeros = torch.zeros_like(target)
                for _ in range(num_user - 1):
                    target = target + apply_gaussian_mechanism(zeros, epsilon, delta, sensitivity)

                train_data = grouped_train_data_dict[key]
                preds_raw, _ = reconstruct_interactions(
                    lambda I: (train(
                        model,
                        params,
                        [
                            (features, ranking, I[indices[idx][0] : indices[idx][1]])
                            for idx, (features, ranking, _) in enumerate(train_data)
                        ],
                        local_lr,
                    )) / local_lr,
                    target / local_lr,
                    indices[-1][1],
                    lr=atk_lr,
                    max_iter=max_iter,
                    num_rounds=num_atk,
                    return_raw=True,
                )
                preds = preds_raw.sigmoid().round().long()
                interactions = torch.cat([I for (_, _, I) in train_data])

                metrics.update(
                    f"{model_name}_{click_model_name}_{num_query}_query_eps_{epsilon}_{num_user}_users_{key}",
                    interactions,
                    preds,
                    preds_raw=preds_raw,
                )

for _ in tqdm(range(num_sim_round)):
    query_ids = data.get_all_query_ids()
    query_ids = random.sample(query_ids, len(query_ids))

    for num_query in num_query_per_user:
        print("Num query", num_query)
        for qids in tqdm(grouper(query_ids, num_query, incomplete="ignore"), total=len(query_ids)//num_query):
            grouped_data = data.get_data_for_queries(list(qids))
            for model_name, model in models.items():
                for click_model_name, click_model in click_models.items():                    
                    simulate_attack(model, model_name, grouped_data, click_model, epsilons, click_model_name, num_query)

metrics.print_summary()
metrics.save("../output/ltr_sec_agg_metrics.csv")

In [ ]:
# Gradient visualization with t-sne
set_seed()

num_query_per_user = 12
num_item_per_ranking = 10
local_lr = 1e-01
num_sim_round = 1000
alphas = [0, 1]

raw_target_dict = {
    alpha: [] for alpha in alphas
}

def train(model, params, grouped_train_data, local_lr=local_lr):
    cur_params = params.clone()

    for features, ranking, interactions in grouped_train_data:
        cur_grad = model.grad(
            cur_params,
            features,
            ranking,
            interactions,
        )

        cur_params = cur_params + local_lr * cur_grad

    return cur_params - params

def simulate_attack(model, model_name, grouped_data, click_model, click_model_name, num_query):
    params = model.gen_params()
    indices = []
    start_ind = 0
    grouped_train_data_dict = {
        alpha: [] for alpha in alphas
    }

    for relevances, features in grouped_data:
        if len(relevances) == 1:
            continue
        features = torch.Tensor(features)
        ranking = model.rank(params, features, sample=True)[:num_item_per_ranking]
        interactions = torch.Tensor(click_model.click(ranking, relevances))
        
        features = features[ranking]
        # Remap the original ranking into the correct range
        _, ranking = torch.where(
            torch.sort(ranking)[0].unsqueeze(1) == ranking.unsqueeze(0)
        )
        num_items = len(ranking)        
        noise = torch.normal(0.0, 0.1, features.shape)
        for alpha in alphas:
            mask = torch.ones_like(features)
            selected_features = random.sample(list(range(num_features)), int(num_features * alpha))
            mask[:, selected_features] = 0.0
            features_adm = mask * features + (1.0 - mask) * noise
            grouped_train_data_dict[alpha].append((features_adm, ranking, interactions))
        
        indices.append((start_ind, start_ind + num_items))
        start_ind += num_items

    if len(grouped_train_data_dict[alphas[0]]) < 1:
        return
    
    for key, train_data in grouped_train_data_dict.items():    
        raw_target_dict[key].append(train(
            model,
            params,
            random.sample(train_data, len(train_data)),
            local_lr,
        ))


query_ids = data.get_all_query_ids()
for _ in tqdm(range(num_sim_round)):
    qids = random.sample(query_ids, num_query_per_user)
    grouped_data = data.get_data_for_queries(list(qids))
    for model_name, model in models.items():
        for click_model_name, click_model in click_models.items():                    
            simulate_attack(model, model_name, grouped_data, click_model, click_model_name, num_query)

num_grads = len(raw_target_dict[0.0])
grads = torch.vstack([
    torch.stack(raw_target_dict[0.0]),
    torch.stack(raw_target_dict[1.0]),
]).numpy()

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 4, figsize=(12, 3))
handles = []  # Collect legend handles
labels = []   # Collect legend labels

for i, perplexity in enumerate([50, 100, 200, 400]):
    visualizer = TSNE(n_components=2, perplexity=perplexity)
    results = visualizer.fit_transform(grads)

    ax = axes[i]
    orange_scatter = ax.scatter(results[:num_grads, 0], results[:num_grads, 1], c="orange", label="No ADM", alpha=1.0, marker='.')
    blue_scatter = ax.scatter(results[num_grads:, 0], results[num_grads:, 1], c="blue", label="ADM", alpha=0.5, marker='.')
    ax.set_title(f'Perplexity = {perplexity}')
    ax.set_xlabel('Component 1')

    if i == 0:
        ax.set_ylabel('Component 2')
        handles.append(orange_scatter)
        handles.append(blue_scatter)
        labels.append("No ADM")
        labels.append("ADM")

lgd = fig.legend(handles, labels, loc="lower center", bbox_to_anchor=(0.5, -0.1), ncols=2)
fig.tight_layout()
fig.savefig("../plots/tsne_mslr10k.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()
